# Patients exploration

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import utils as mutil
import sklearn
import sklearn.ensemble
import sklearn.model_selection

from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

import tabulate
def print_table(df):
    # Print the stuff
    data = [["Column", "dtype", "# NaN"]]
    for t, c in zip(df.dtypes, df.columns.values):
        if t == 'object':
            print("\x1b[31m", t, "\t", c, "\x1b[0m")
        # print(t, "\t", c)
        data.append([c, t, df[c].isna().sum()])

    table = tabulate.tabulate(data, tablefmt='html')
    display(table)

## Load dataset

In [2]:
patients_ds = pd.read_csv("patient/dataset.csv")
patients_ds = patients_ds.drop(columns=["Unnamed: 83"])

## Layouttitanic_train.head()

In [3]:
patients_ds.head()


,encounter_id,patient_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,icu_admit_source,...,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem,hospital_death
0,66154,25312,118,68.0,22.73,0,Caucasian,M,180.3,Floor,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular,0
1,114252,59342,81,77.0,27.42,0,Caucasian,F,160.0,Floor,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Respiratory,Respiratory,0
2,119783,50777,118,25.0,31.95,0,Caucasian,F,172.7,Accident & Emergency,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Metabolic,Metabolic,0
3,79267,46918,118,81.0,22.64,1,Caucasian,F,165.1,Operating Room / Recovery,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Cardiovascular,Cardiovascular,0
4,92056,34377,33,19.0,NaN,0,Caucasian,M,188.0,Accident & Emergency,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Trauma,Trauma,0


In [4]:
patients_ds.describe(include='all')

,encounter_id,patient_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,icu_admit_source,...,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem,hospital_death
count,91713.000000,91713.000000,91713.000000,87485.000000,88284.000000,91713.000000,90318,91688,90379.000000,91601,...,90998.000000,90998.000000,90998.000000,90998.000000,90998.000000,90998.000000,90998.000000,90051,90051,91713.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,6,2,NaN,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,10,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,Caucasian,M,NaN,Accident & Emergency,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cardiovascular,Cardiovascular,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,70684,49469,NaN,54060,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29999,38816,NaN
mean,65606.079280,65537.131464,105.669262,62.309516,29.185818,0.183736,NaN,NaN,169.641588,NaN,...,0.015693,0.225192,0.012989,0.026165,0.007066,0.004132,0.020638,NaN,NaN,0.086302
std,37795.088538,37811.252183,62.854406,16.775119,8.275142,0.387271,NaN,NaN,10.795378,NaN,...,0.124284,0.417711,0.113229,0.159628,0.083763,0.064148,0.142169,NaN,NaN,0.280811
min,1.000000,1.000000,2.000000,16.000000,14.844926,0.000000,NaN,NaN,137.200000,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,0.000000
25%,32852.000000,32830.000000,47.000000,52.000000,23.641975,0.000000,NaN,NaN,162.500000,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,0.000000
50%,65665.000000,65413.000000,109.000000,65.000000,27.654655,0.000000,NaN,NaN,170.100000,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,0.000000
75%,98342.000000,98298.000000,161.000000,75.000000,32.930206,0.000000,NaN,NaN,177.800000,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,0.000000


In [5]:
# print("Columns:", patients_ds.columns.values)

# Convert Categorical to numeric
for t, c in zip(patients_ds.dtypes, patients_ds.columns.values):
    if t == 'object':
        print("\x1b[31m", t, "\t", c, "\x1b[0m")
        patients_ds[c] = pd.Categorical(patients_ds[c])
        patients_ds[c] = patients_ds[c].cat.codes

# Convert problematic types
float64_cols = list(patients_ds.select_dtypes(include='float64'))
patients_ds[float64_cols] = patients_ds[float64_cols].astype('float32')

int64_cols = list(patients_ds.select_dtypes(include='int64'))
patients_ds[int64_cols] = patients_ds[int64_cols].astype('int32')

print_table(patients_ds)

 object 	 ethnicity 
 object 	 gender 
 object 	 icu_admit_source 
 object 	 icu_stay_type 
 object 	 icu_type 
 object 	 apache_3j_bodysystem 
 object 	 apache_2_bodysystem 


Column,dtype,# NaN
encounter_id,int32,0
patient_id,int32,0
hospital_id,int32,0
age,float32,4228
bmi,float32,3429
elective_surgery,int32,0
ethnicity,int8,0
gender,int8,0
height,float32,1334
icu_admit_source,int8,0


In [6]:

        
def replace_random(df, c):
    avg = df[c].mean()
    std = df[c].std()
    null_count = df[c].isnull().sum()
    
    null_random_list = np.random.randint(avg - std, avg + std, size=null_count)
    df[c][np.isnan(df[c])] = null_random_list
    
def replace_zero(df, c):
    df[c] = df[c].fillna(0)

# To keep the distribution properties, we replace NaN in these columns by distribution preserving random values
random_replace_columns = [ "age", "bmi", "height","weight" ]
# Having no information on these values can have semantic meaning therefore we set them to 0 (in essence introducing an unknown class)
zero_replace_columns = [
    "apache_2_diagnosis",
    "apache_3j_diagnosis",
    "arf_apache",
    "gcs_eyes_apache",
    "gcs_motor_apache",
    "gcs_unable_apache",
    "gcs_verbal_apache",
    "heart_rate_apache",
    "intubated_apache",
    "map_apache",
    "resprate_apache",
    "temp_apache",
    "ventilated_apache",
    "d1_diasbp_max",
    "d1_diasbp_min",
    "d1_diasbp_noninvasive_max",
    "d1_diasbp_noninvasive_min",
    "d1_heartrate_max",
    "d1_heartrate_min",
    "d1_mbp_max",
    "d1_mbp_min",
    "d1_mbp_noninvasive_max",
    "d1_mbp_noninvasive_min",
    "d1_resprate_max",
    "d1_resprate_min",
    "d1_spo2_max",
    "d1_spo2_min",
    "d1_sysbp_max",
    "d1_sysbp_min",
    "d1_sysbp_noninvasive_max",
    "d1_sysbp_noninvasive_min",
    "d1_temp_max",
    "d1_temp_min",
    "h1_diasbp_max",
    "h1_diasbp_min",
    "h1_diasbp_noninvasive_max",
    "h1_diasbp_noninvasive_min",
    "h1_heartrate_max",
    "h1_heartrate_min",
    "h1_mbp_max",
    "h1_mbp_min",
    "h1_mbp_noninvasive_max",
    "h1_mbp_noninvasive_min",
    "h1_resprate_max",
    "h1_resprate_min",
    "h1_spo2_max",
    "h1_spo2_min",
    "h1_sysbp_max",
    "h1_sysbp_min",
    "h1_sysbp_noninvasive_max",
    "h1_sysbp_noninvasive_min",
    "d1_glucose_max",
    "d1_glucose_min",
    "d1_potassium_max",
    "d1_potassium_min",
    # "apache_4a_hospital_death_prob",
    # "apache_4a_icu_death_prob",
    "aids",
    "cirrhosis",
    "diabetes_mellitus",
    "hepatic_failure",
    "immunosuppression",
    "leukemia",
    "lymphoma",
    "solid_tumor_with_metastasis",
]

# Remove the Appache III score for testing
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4893757/
patients_ds = patients_ds.drop(columns=["apache_4a_hospital_death_prob", "apache_4a_icu_death_prob"])

for c in random_replace_columns:
    replace_random(patients_ds, c)

for c in zero_replace_columns:
    replace_zero(patients_ds, c)

print_table(patients_ds)

C:\Users\flori\AppData\Local\Temp\ipykernel_24572\3375241567.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[c][np.isnan(df[c])] = null_random_list


Column,dtype,# NaN
encounter_id,int32,0
patient_id,int32,0
hospital_id,int32,0
age,float32,0
bmi,float32,0
elective_surgery,int32,0
ethnicity,int8,0
gender,int8,0
height,float32,0
icu_admit_source,int8,0


# Notes:
* maybe to complex, switch to https://www.kaggle.com/datasets/saurabhshahane/in-hospital-mortality-prediction ???

## Train, Test, Validation split

In [7]:
train, val, test = mutil.split_train_val_test(patients_ds, validation_size=0.1, test_size=0.2)

In [8]:
# No missing labels
train['hospital_death'].isna().sum()

0

In [14]:
# Create training dataset
cross_folds = sklearn.model_selection.KFold(n_splits=5)
parameter_set = [{'max_depth': d} for d in range(1, 10)] # 1 to number of attributes + 1 

accuracies = []

for ps in parameter_set:
    print(f"Running for parameter set {ps} ...")
    model = sklearn.tree.DecisionTreeClassifier(**ps, class_weight="balanced")
    accuracies.append([])
    for train_fold, validation_fold in cross_folds.split(patients_ds):
        # Create training dataset for fold
        train = patients_ds.loc[train_fold]
        train_x = train.drop(columns=['hospital_death'])
        train_y = train['hospital_death']
        # Create validation dataset for fold
        valid = patients_ds.loc[validation_fold]
        valid_x = valid.drop(columns=['hospital_death'])
        valid_y = valid['hospital_death']
        # Train model
        model = model.fit(X=train_x, y=train_y)
        # Calculate accuracy
        accuracy = model.score(X=valid_x, y=valid_y)
        accuracies[-1].append(accuracy)

for i, acc in enumerate(accuracies):
    print(f"Average of fold  with params {parameter_set[i]} is {sum(acc)/len(acc)} full data:\n\t {acc}")

# The best depth seems to be 4


Running for parameter set {'max_depth': 1} ...
Running for parameter set {'max_depth': 2} ...
Running for parameter set {'max_depth': 3} ...
Running for parameter set {'max_depth': 4} ...
Running for parameter set {'max_depth': 5} ...
Running for parameter set {'max_depth': 6} ...
Running for parameter set {'max_depth': 7} ...
Running for parameter set {'max_depth': 8} ...
Running for parameter set {'max_depth': 9} ...
Average of fold  with params {'max_depth': 1} is 0.7057672930215112 full data:
	 [0.7358120263860873, 0.6720819931308947, 0.6736629777026659, 0.7354705048522516, 0.7118089630356559]
Average of fold  with params {'max_depth': 2} is 0.6903056064297927 full data:
	 [0.7280161369459739, 0.6492394919042687, 0.6955786948699777, 0.7127357976229419, 0.6659579108058009]
Average of fold  with params {'max_depth': 3} is 0.7626402643866019 full data:
	 [0.7976339748132802, 0.6966690290574061, 0.7670501008559123, 0.7804492421764256, 0.7713989750299858]
Average of fold  with params {'

In [21]:
# Create training dataset
depth = 8
train_x = train.drop(columns=['hospital_death'])
train_y = train['hospital_death']
# Train model based on best estimated configuration from previous tests
model = sklearn.tree.DecisionTreeClassifier(max_depth=depth, class_weight="balanced")
model = model.fit(X=train_x, y=train_y)
# Calculate train accuracy
train_accuracy = model.score(X=train_x, y=train_y)
print(f"Train accuracy {train_accuracy}")


Train accuracy 0.7722942306905999


In [22]:
# Create validation dataset
val_x = val.drop(columns=['hospital_death'])
val_y = val['hospital_death']
# Calculate train accuracy
val_accuracy = model.score(X=val_x, y=val_y)
print(f"Validation accuracy {val_accuracy}")


Validation accuracy 0.7771478412559966


In [23]:
# Create test dataset
test_x = test.drop(columns=['hospital_death'])
test_y = test['hospital_death']
# Calculate train accuracy
test_accuracy = model.score(X=test_x, y=test_y)
print(f"Test accuracy {test_accuracy}")


Test accuracy 0.7635065147467699


In [24]:
tree_graph = sklearn.tree.export_graphviz(model,
                filled=True, rounded=True,
                  special_characters=True, 
                  feature_names = patients_ds.columns[1:],
                  class_names=['Died','Survived'])
with open(f"tmp-pat-tree-d{depth}-graphviz.graph", 'w') as f:
    f.write(tree_graph)

### Most important feature according to descision tree

In [25]:
# Create training dataset
depth = 1
train_x = train.drop(columns=['hospital_death'])
train_y = train['hospital_death']
# Train model based on best estimated configuration from previous tests
model = sklearn.tree.DecisionTreeClassifier(max_depth=depth, class_weight="balanced")
model = model.fit(X=train_x, y=train_y)
# Calculate train accuracy
train_accuracy = model.score(X=train_x, y=train_y)
print(f"Train accuracy {train_accuracy}")

# Create test dataset
test_x = test.drop(columns=['hospital_death'])
test_y = test['hospital_death']
# Calculate train accuracy
test_accuracy = model.score(X=test_x, y=test_y)
print(f"Test accuracy {test_accuracy}")

tree_graph = sklearn.tree.export_graphviz(model,
                filled=True, rounded=True,
                  special_characters=True, 
                  feature_names = patients_ds.columns[1:],
                  class_names=['Died','Survived'])
with open(f"tmp-pat-tree-d{depth}-graphviz.graph", 'w') as f:
    f.write(tree_graph)

Train accuracy 0.7042564500960871
Test accuracy 0.7055007359755765


In [ ]:
mutil.plot_roc(model, test_x, test_y, title="ROC curve for best fit model")